In [1]:
from time import sleep
from random import random
import pandas as pd
import requests
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.parse as urlparse
from urllib.parse import parse_qs

In [2]:
# Constants
BASE_URL = 'https://www.flipkart.com/'
SEARCH_QUERY = "books"
TOP_N_PRODUCTS = 10
REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT = 2000 #10 Reviews exist per page

In [3]:
SAMPLE_URL = "https://www.flipkart.com/boat-rockerz-255f-bluetooth-headset/product-reviews/itmd35e57aca99fb?pid=ACCF6SZ8EFWFEPZ6&lid=LSTACCF6SZ8EFWFEPZ6M21QA4&sortOrder=MOST_HELPFUL&certifiedBuyer=false&aid=overall"
r = requests.get(SAMPLE_URL)    
soup = BeautifulSoup(r.content, 'html.parser') 
print(soup.prettify()[:500])

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="https://rukminim1.flixcart.com" rel="preconnect"/>
  <link href="//static-assets-web.flixcart.com/www/linchpin/fk-cp-zion/css/app.chunk.2bdc3d.css" rel="stylesheet"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="102988293558" property="fb:page_id"/>
  <meta content="658873552,624500995,100000233612389" property="fb:admins"/>
  <meta content="n


In [6]:
# Extracting all review blocks
## Note col._2wzgFH.K0kLPL means 3 entities namely 'col', ' _2wzgFH' and 'K0kLPL' 
## This is written in HTML as 'col _2wzgFH K0kLPL'
## This can also be seen in Bullet 3

rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})
print(f"Count of rows(reviews):{len(rows)}\n\n\n")
# iteration over all blocks
for row in rows:
    # Print a sample row(review html block)
    # print(f"row:\n{row} \n\n")
    
    # finding all rows within the block
    sub_row = row.find_all('div',attrs={'class':'row'})
        
    # extracting text from 1st and 2nd row
    rating = sub_row[0].find('div').text
    print(f"rating:{rating} \n\n")
    
    summary = sub_row[0].find('p').text
    print(f"summary:{summary} \n\n")
    
    review = sub_row[1].find_all('div')[2].text
    print(f"review:{review} \n\n")
    
    location = sub_row[3].find('p',attrs={'class':'_2mcZGG'}).find_all('span')[1].text
    location = "".join(location.split(",")[1:]).strip()
    print(f"location:{location} \n\n")
    
    date = sub_row[3].find_all('p',attrs={'class':'_2sc7ZR'})[1].text
    print(f"date:{date} \n\n")
    
    
    sub_row_2 = row.find_all('div',attrs={'class':'_1e9_Zu'})[0].find_all('span',attrs={'class':'_3c3Px5'})
    
    upvotes = sub_row_2[0].text
    print(f"upvotes:{upvotes} \n\n")
    
    downvotes = sub_row_2[1].text
    print(f"downvotes:{downvotes} \n\n")
    
    break

Count of rows(reviews):10



rating:5 


summary:Best in the market! 


review:Amazing sound quality with boosted Bass. But if your r listing music on YouTube then might the sound quality may not that good as u experience on Gaana aap or related music app or music that you have download on your cell phone. Don't worry for that it only differ slightly i.e. unpredictable. Overall you will get owsm sound and bass experiences. Coming to the battery backup it is up to 6hrs + and charges quickly for urgent work.The build quality is good and has a magnetic end that don't work... 


location:Gaya 


date:Jun, 2020 


upvotes:2596 


downvotes:397 




In [5]:
## Website Links

In [6]:
link = ['https://www.flipkart.com/five-point-someone-not-do-iit/product-reviews/itmffugyyxyszfyn?pid=9788129135490&lid=LSTBOK9788129135490OPMQY4&marketplace=FLIPKART',
        'https://www.flipkart.com/one-night-call-centre/product-reviews/itm35f1b52eb53e9?pid=9788129135506&lid=LSTBOK9788129135506OXUV39&marketplace=FLIPKART',
        'https://www.flipkart.com/3-mistakes-my-life/product-reviews/itmf0a18d6b138e8?pid=9788129135513&lid=LSTBOK9788129135513L0JG94&marketplace=FLIPKART',
        'https://www.flipkart.com/2-states/product-reviews/itmffugyxzh2bajb?pid=9788129115300&lid=LSTBOK9788129115300VXJ1XR&marketplace=FLIPKART',
        'https://www.flipkart.com/half-girlfriend/product-reviews/itmffugy6f7yvcdt?pid=9789551718114&lid=LSTBOK97895517181145ZWBZW&marketplace=FLIPKART',
        'https://www.flipkart.com/half-girlfriend/product-reviews/itm309299e5fd828?pid=9788129124029&lid=LSTBOK9788129124029N6KB0G&marketplace=FLIPKART',
        'https://www.flipkart.com/one-indian-girl/product-reviews/itm97bff2ed4a5b2?pid=9788129142146&lid=LSTBOK9788129142146N3YMFX&marketplace=FLIPKART',
        'https://www.flipkart.com/girl-room-105/product-reviews/itm162e2372adf06?pid=9781542040464&lid=LSTBOK9781542040464SHRSPR&marketplace=FLIPKART',
        'https://www.flipkart.com/one-arranged-murder/product-reviews/itmb4cd0197e98ce?pid=9781542094139&lid=LSTBOK9781542094139A5FY6R&marketplace=FLIPKART'
       ]

In [7]:
## Web Scraping from Flipkart

In [ ]:
# DS-1

In [11]:
link = ['https://www.flipkart.com/five-point-someone-not-do-iit/product-reviews/itmffugyyxyszfyn?pid=9788129135490&lid=LSTBOK9788129135490OPMQY4&marketplace=FLIPKART']

In [ ]:
dataset_1 = []

for idx, url in enumerate(tqdm(link, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(random())
        soup = BeautifulSoup(r.content, 'html.parser') 
        
        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()

            # appending to data
            dataset_1.append({'product_id':pid, 'rating': rating, 'summary': summary, 'review': review})

In [13]:
df = pd.DataFrame(dataset_1)

with pd.option_context('display.max_colwidth', -1):
    display(df.head(5))
    display(df.tail(5))

,product_id,rating,summary,review
0,9788129135490,2,Dime novel through and through,"I first read it a good six years ago and had loved it. I still like it. It is no literary masterpiece and it would be wrong to compare it with one. It is a dime novel and a good one. It offers a good, quick read - it is a page turner and keeps you hooked till the end. Good pieces of literature linger with you even when you have read the last word. This is not one of them. It does not leave any lingering impressions. You do feel a little good and warm when you reach the end. If you have to wh..."
1,9788129135490,5,I WOULD LOVE TO READ IT AGAIN AND AGAIN AS IT REMINDS ME ABOUT MY COLLEGE DAYS,ACTUALLY I READ THE BOOK WHEN IT WAS RELEASED FIRST AND I REPEAT IT TO REGAIN MY COLLEGE DAYS MEMORIES AND IT IS TRUE STORY FOR EVERY STUDENT LIKE ME WHO HAD DONE OR DOING ENGINEERING AS WE EXPERIENCED THE SAME ............I LOST MY COPY OF THIS BOOK SO WHEN I SAWED THIS ON YOUR SITE AT REASONABLE RATE SO I ORDERED IT..........I THANK U FOR MAKING ME HAPPY BY PROVIDING THIS.........AND I REALLY THANK U FOR THE FAST DELIVERY...........
2,9788129135490,5,Brilliant,"Absolutely the top of the shelf. 3 Idiots, the movie based on the book, despite all the money it made, was not a patch on the book. Sincere, honest, witty and poignant at the same time, the boo will assure Chetan Bhagat a place in Indian literary history. read it if you have not done so already."
3,9788129135490,3,3 idiots better than the book!!!,"This is a great book for IITans.THe book has a message on it's own while 3 idiots is better than the book.Things that Raj Kumar Hirani has changed in his movie:-Ryan=Rancho,Alok=Raju,Hari=Farhan,Neha=Priya,Virus=Prof. Cherian,Chatur=Venkat(Who plays a minor role).1.Farhan is in love with Priya not Rancho.2.Alok wants to become an artist,While in the movie Farhan wants to and becomes a photographer.3.The thing that Rancho is Phunsukh Wangdu is not at all mentioned in the book.Not only ..."
4,9788129135490,5,Super!,"Exceptionally well.The book needs no introduction.This books tells all the naughty and productive things an engineer does in his engineering.A pure combination of comedy, tragedy, love and friendship.The original of 3 Idiots."


,product_id,rating,summary,review
592,9788129135490,3,Does the job,"Not very satisfied not with the product but with the price,with delivery charges there is no point in the discount.It's basically the same as the original price"
593,9788129135490,3,Satisfied...,Quick delivery....within 24 hrs :)
594,9788129135490,4,good book,great book .but its different from the movie..........................................................................................................................................................
595,9788129135490,5,good,good delivery service good book everything good!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
596,9788129135490,5,not started reading...but great delivery with nice condition,"I dont need to say anything about the book. As per cover and book condition, i just got the book in 4 days and it is not late as I am living in Cuttack. And condition is great and I dont think u will find any other place with such a good price"


In [14]:
#df.to_csv('/Users/kritya/Desktop/College/SEM 7/Project Phase 1/DS/FYP - FlipBook1.csv')

In [ ]:
# DS-2

In [7]:
link = ['https://www.flipkart.com/one-night-call-centre/product-reviews/itm35f1b52eb53e9?pid=9788129135506&lid=LSTBOK9788129135506OXUV39&marketplace=FLIPKART']
dataset_2 = []

for idx, url in enumerate(tqdm(link, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(random())
        soup = BeautifulSoup(r.content, 'html.parser') 
        
        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()

            # appending to data
            dataset_2.append({'product_id':pid, 'rating': rating, 'summary': summary, 'review': review})

In [8]:
df2 = pd.DataFrame(dataset_2)

with pd.option_context('display.max_colwidth', -1):
    display(df2.head(5))
    display(df2.tail(5))

,product_id,rating,summary,review
0,9788129135506,4,Worth the money,"Excellent, same as author i was thinking how a person can write a book on 'call centre'. not 5 stars bcz i feel there are some gags. overall it's fabulous. i Love this page where chetan sir talking about their twin whose created by his better half👌 and before reading complete the exercise it's amazing idea."
1,9788129135506,4,Good quality product,"A beautifully knitted plot. Thoroughly enjoyable. This is Chetan Bhagat's undeniable quality. You won't be bored. But to me his work is like a bollywood movie where everything will be fine at the end. Though life does not work like that. His work lacks this. But '2 States' & ""Revolution 20/20' are different."
2,9788129135506,5,Worth every penny,"This is still my favorite book, and the first book i read. Was really well written and good story. Though there are some fiction beyond human things, it is worth the read. Will make you understand life.. Lots of small messages in this book, if you understand the depth of the story. A must read."
3,9788129135506,3,Decent product,"A bit monotonous at the beginning but very interesting at the end, gives better opinions for enjoying a good life...overall not bad."
4,9788129135506,5,Fabulous!,Just completed reading this fantastic book... Hats off to Mr. CHETAN BHAGAT ... don't have words to explain how good & positivity the book has brought to my life..just love it... Go for it guys.. just gram it with both hands... Happy reading... Thank you Chetan sir for mesmerising me with your detailed script.... Love u


,product_id,rating,summary,review
410,9788129135506,5,Classy product,"Wonderful service,flipkart!!....i love ur service...i got my book within five days after the order...my book is just awsome..one night at the call centre by chethan bhagat is fantastic...it is absolutely breath taking.......love it!!!!!!"
411,9788129135506,5,Just wow!,Good service
412,9788129135506,5,Mind-blowing purchase,Superb service till now.
413,9788129135506,5,Wonderful,"Delivered the product on time :) even before the planned date.. Nice packing, .... Review about the book is subjective... For me its a masterpiece of chetan sir, value for the money? go on ...."
414,9788129135506,4,Nice!!,Nice bookBut not chetans best.It certainly could hav been more racy at some times!


In [9]:
df2.to_csv('/Users/kritya/Desktop/College/SEM 7/Project Phase 1/DS/FYP - FlipBook2.csv')

In [ ]:
# DS-3

In [11]:
link = ['https://www.flipkart.com/3-mistakes-my-life/product-reviews/itmf0a18d6b138e8?pid=9788129135513&lid=LSTBOK9788129135513L0JG94&marketplace=FLIPKART']
dataset_3 = []

for idx, url in enumerate(tqdm(link, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,1000+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(random())
        soup = BeautifulSoup(r.content, 'html.parser') 
        
        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()

            # appending to data
            dataset_3.append({'product_id':pid, 'rating': rating, 'summary': summary, 'review': review})

In [12]:
df3 = pd.DataFrame(dataset_3)

with pd.option_context('display.max_colwidth', -1):
    display(df3.head(5))
    display(df3.tail(5))

,product_id,rating,summary,review
0,9788129135513,5,The Most Knowledgable & Amazing Book,I have chosen this product because I had actually liked the product and I thought that I could gain some kind of knowledge and gain for myself if I had read and understood this book very well and even my knowledge and will-power would have been increased by trying to learn and read through this book.
1,9788129135513,4,Nice product,"A good book from Chetan Bhagat.. Different from other books but I felt a bit similar to Five point someone, the friendship and love part.. A good option to try."
2,9788129135513,4,Really Nice,this book is amazing !! I love this book Chetan bhagat books are always amazing and interesting...I love this book ..💛
3,9788129135513,5,wow,wow that’s a wonderful book I ever read... the books are in perfect shape even without a scratch even though they were delivered from a far place ... superb courier service and good book
4,9788129135513,5,Classy product,"Best book, amazing content. I just loved it. Thank you Chetan Bhagat for such an amazing content."


,product_id,rating,summary,review
808,9788129135513,5,Classy product,Deliver on timeThank you
809,9788129135513,5,Perfect product!,Thank you
810,9788129135513,5,Packing,Good packing
811,9788129135513,5,Good one,"Got in time,well packed, new product.loved it"
812,9788129135513,2,Cheap quality,The quality of the book pages are bad and even worst but good timely from flipkart ðÂ?Â?Â?


In [13]:
df3.to_csv('/Users/kritya/Desktop/College/SEM 7/Project Phase 1/DS/FYP - FlipBook3.csv')

In [ ]:
# DS-4

In [19]:
link = ['https://www.flipkart.com/2-states/product-reviews/itmffugyxzh2bajb?pid=9788129115300&lid=LSTBOK9788129115300VXJ1XR&marketplace=FLIPKART']
dataset_4 = []

for idx, url in enumerate(tqdm(link, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,1000+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(1)
        soup = BeautifulSoup(r.content, 'html.parser') 
        
        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()

            # appending to data
            dataset_4.append({'product_id':pid, 'rating': rating, 'summary': summary, 'review': review})

In [20]:
df4 = pd.DataFrame(dataset_4)

with pd.option_context('display.max_colwidth', -1):
    display(df4.head(5))
    display(df4.tail(5))

,product_id,rating,summary,review
0,9788129115300,3,2 States (Book) - Chetan Bhagat,"The book and the story: As the top of the cover itself says “2 States – The story of my marriage” – you can imagine what it’s all about. I still have no idea about him and his life yet but but I can tell you one thing about him after reading this book that I guess this is his own life story and I am sure Chetan Bhagat is one hopelessly romantic guy, who is madly in love with his wife – so much so that he just can’t stop bragging about her and this book is nothing but the same story told in ..."
1,9788129115300,5,MUST READ FOR LOVERS...,"I am a real diehard fan of ChetanBhagat. He is my favorite writer. His books are my favorite.. ""2states"" the most favorite of them all.. 2States is one of the best love stories I have ever heard or read about. A real transpiring love story.. I read the book in one nonstop sitting. Completed reading it for the eighth time now and I've never got bored reading it.. The book is an eye opener for young minds.For someone who thought the journey from start of 'true love' towards tying the knot w..."
2,9788129115300,5,"A very good book, no strings attached","Well, i had heard good things about this book and that is when i finally decided to buy itIt started a bit slow and i was always trying to mentally compare it to ""5 point someone"" so i was initially a little disappointed but Boy, did i know what was coming up next, as chapters passed i got so much into the storyline that i did not even have track of time and read the book through the night to finish itThe key points that i liked is1. Simple and lively language2. A potent mixture of humor ..."
3,9788129115300,5,It's fabulous and highly recommended,I would suggest every romantic tale lover to read this book as it's a good eg. Of true love
4,9788129115300,4,the book which got me footing in the world of english,"well..........'jab kuch achha lagta hai to vo mushkil hoke bhi asaan ho jaata hai!'bought this novel from a flee market with purpose of learning english as advised by someone and to my amazement i found it intriguing me to read further.as i was a beginner so it took me 1 month to go through it.for me it was a wonderful experience to learn as well as delight.In the country like India where love story is the best amusement being provided by yashraj films, Chetan sir also served the same & he..."


,product_id,rating,summary,review
605,9788129115300,5,Great Serving,Delivers in 1 day...Awesome Book Also
606,9788129115300,4,Super novel,Thanks for flipkart delivered on time.Story is awesome thanks of chethan giving such a novel.I suggest to Buyers to buy this novel as early as possible
607,9788129115300,5,Good,"Good Excellent in condition as described, i am very happy with flipkart delivery. Price is also reasonable."
608,9788129115300,5,Good read !!,overall nice read. Product received in fantastic order.................................kudos flipkart
609,9788129115300,1,very bad,Its realy very disappointing thing that flipkart sold a secondhand book and very old book.. I'm realy not satisfied with this..


In [21]:
df4.head()

,product_id,rating,summary,review
0,9788129115300,3,2 States (Book) - Chetan Bhagat,The book and the story: As the top of the cov...
1,9788129115300,5,MUST READ FOR LOVERS...,I am a real diehard fan of ChetanBhagat. He is...
2,9788129115300,5,"A very good book, no strings attached","Well, i had heard good things about this book ..."
3,9788129115300,5,It's fabulous and highly recommended,I would suggest every romantic tale lover to r...
4,9788129115300,4,the book which got me footing in the world of ...,well..........'jab kuch achha lagta hai to vo ...


In [22]:
df4.to_csv('/Users/kritya/Desktop/College/SEM 7/Project Phase 1/DS/FYP - FlipBook4.csv')

In [4]:
# DS-5

In [7]:
link = ['https://www.flipkart.com/half-girlfriend/product-reviews/itmffugy6f7yvcdt?pid=9789551718114&lid=LSTBOK97895517181145ZWBZW&marketplace=FLIPKART',
        'https://www.flipkart.com/half-girlfriend/product-reviews/itm309299e5fd828?pid=9788129124029&lid=LSTBOK9788129124029N6KB0G&marketplace=FLIPKART']
dataset_5 = []

for idx, url in enumerate(tqdm(link, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,2000+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(1)
        soup = BeautifulSoup(r.content, 'html.parser') 
        
        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()

            # appending to data
            dataset_5.append({'product_id':pid, 'rating': rating, 'summary': summary, 'review': review})

KeyboardInterrupt: 

In [8]:
df5 = pd.DataFrame(dataset_5)

with pd.option_context('display.max_colwidth', -1):
    display(df5.head(5))
    display(df5.tail(5))

,product_id,rating,summary,review
0,9789551718114,5,Worth every penny,Totally loved it .. You are a fabulous writer sir .. It's really a magnificent story of unconditional love .. At last it made me cry when Madhav found Riya .. It was my best book purchase ever ..
1,9789551718114,5,Worth every penny,Never read such a nice story. Loved the book. Everybody must love like Madhav Jha in life and get a prettiest girl like Riya Somani. Lovely story !!!!
2,9789551718114,5,Mind-blowing purchase,"Thank you flipkart. I love this book 📙. I want to see half girlfriend movie bt I can't get it from anywhere finally I bought this through flipkart, read it with only 2and a half day bcoz of my curiosity for half girlfriend. Apart from that, first time I read a book of chetan bhagat. I want to enrich my English language and vocabulary so I preffered mostly English books, before this I bought the book 3 men in a boat by Jerome K Jerome. I don't like that book bcoz of its bad content felt me v..."
3,9789551718114,1,Horrible,Delivery charge is very high.
4,9789551718114,5,Classy product,One real massive luv storyI can assure that no one can express a simple luv story better than chetan


,product_id,rating,summary,review
3727,9788129124029,4,Just sayin',"I was one of the first to go for the pre-order of this book, still I haven't got a signed copy as was the offer when I ordered the book. The delivery was right on time, everything was good. But, the thing for which I ordered so early went in vain... What happened flipkart? How many copies did he sign himself?"
3728,9788129124029,3,disappointed,autographed copies for customers who pre order was promised. i didnt get it the autographed one. packing is just normal . homeshop 18 is better. no book marks nothing.3 stars jus for the book and the author. nothing for flipkart.
3729,9788129124029,4,20 Pages MISSING?????,"Received the book...but it seems 20 pages are missing. Book i recd have 260 pages whereas flipkart show Length: 280 pages. Not yet read the book, so rating's are like that only..."
3730,9788129124029,2,AUTHOR AUTOGRAPH IS ABSENT ON PRE-ORDERED COPIES,FLIPKART SAID PRE-ORDERED BOOKS ARE AUTOGRAPHED BY AUTHOR CHETAN BHAGAT.. I RECIEVED WITH BLANK...DISSAPPOINTED COZ NOT UP FOR WORD..
3731,9788129124029,3,Book is good.. Flipkart not!,"Had pre-ordered this book way long back in August, the day it was announced. Flipkart had also mentioned that the first pre orders would received signed copies by Chetan Bhagat. Guess what, mine arrived yesterday and it's not a signed copy!! This sucks!About the book: it's a very good read just like every other Chetan Bhagat book.. I started reading it yesterday noon and was hooked to it and completed reading the book by midnight with few breaks in between.. It's a must read! Go ahead and b..."


In [9]:
df5.to_csv('/Users/kritya/Desktop/College/SEM 7/Project Phase 1/DS/FYP - FlipBook5.csv')

In [ ]:
# DS-6

In [4]:
link = ['https://www.flipkart.com/one-indian-girl/product-reviews/itm97bff2ed4a5b2?pid=9788129142146&lid=LSTBOK9788129142146N3YMFX&marketplace=FLIPKART']
dataset_6 = []

for idx, url in enumerate(tqdm(link, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,5000+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(2)
        soup = BeautifulSoup(r.content, 'html.parser') 
        
        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()

            # appending to data
            dataset_6.append({'product_id':pid, 'rating': rating, 'summary': summary, 'review': review})

KeyboardInterrupt: 

In [5]:
df6 = pd.DataFrame(dataset_6)

with pd.option_context('display.max_colwidth', -1):
    display(df6.head(5))
    display(df6.tail(5))

df6.to_csv('/Users/kritya/Desktop/College/SEM 7/Project Phase 1/DS/FYP - FlipBook6.csv')

,product_id,rating,summary,review
0,9788129142146,5,Brilliant,"Firstlly chetan sir , i was so excited for the book and my excitment gets double n triple with every next chapter i turned. ..i felt its me....radhika character & her decision is what i did just a bit different...thank you for such a beautiful writings ...looking forward to your movie...all the best."
1,9788129142146,3,Read it if u r a girl or if u r man enough to read about a n,Radhika Mishra is One Indian girl who is intelligent and smart. She works for Goldman Sacks in a foreign land and earns a lot of money. She has had two boyfriends – a creative Bengali boy (Debashish Sen) and a married boss (Neel Gupta) who is two decades older. But Radhika is not lucky in love. She breaks up with both of them for almost similar reasons (don’t ask me what it is because it could be a spoiler). She finally gives in for family pressure and settles for arranged marriag...
2,9788129142146,1,Expected a better product,"Dear Chetan Bhagat this book story is very good thats a good point, but i rated it 2 star because we don't want a film script every time coz this story will be a script of another upcoming bollywood movie so u better write film scripts only n start producing films also that will be a better buiness. No more film scripts in a book format plz n one more request plz avoid sex adventures in your next film script coz its a common part which we find in your scripts. And this book story can be easi..."
3,9788129142146,1,Very predictable story,"My first book of the writer was ""2 states"". I instantly became a fan of him. Next was ""3mistakes of my life"". It was also good. But in case of "" R2020"" , ""half gf"" and this book I personally feel that the writer has not anything left new to share with his readers. The same writing style-the same way to narrate a story it felt like I'm walking down the same road. And about feminism-the USP of the book--I don't think that it works in the way as it shows!"
4,9788129142146,5,Terrific purchase,It is really a nice book.... thanks to Chetan Sir...Delivery was excellent...I got it in 1 day advance...The book came in very good condition.... fully seal packed... Best of luck Flipkart


,product_id,rating,summary,review
4567,9788129142146,5,Excellent,I really love flipkart!!!!!
4568,9788129142146,5,Just wow!,Just Started Reading ... Delivery agency great service
4569,9788129142146,5,Fabulous!,Nice packing and on time delivery....Chetan again made it
4570,9788129142146,5,Excellent,Got it for rs.1..........Thank you flipkart.
4571,9788129142146,5,Terrific,Quick service.


In [ ]:
# DS-7

In [6]:
link = ['https://www.flipkart.com/girl-room-105/product-reviews/itm162e2372adf06?pid=9781542040464&lid=LSTBOK9781542040464SHRSPR&marketplace=FLIPKART']
dataset_7 = []

for idx, url in enumerate(tqdm(link, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,5000+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(2)
        soup = BeautifulSoup(r.content, 'html.parser') 
        
        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()

            # appending to data
            dataset_7.append({'product_id':pid, 'rating': rating, 'summary': summary, 'review': review})

KeyboardInterrupt: 

In [7]:
df7 = pd.DataFrame(dataset_7)

with pd.option_context('display.max_colwidth', -1):
    display(df7.head(5))
    display(df7.tail(5))

df7.to_csv('/Users/kritya/Desktop/College/SEM 7/Project Phase 1/DS/FYP - FlipBook7.csv')

,product_id,rating,summary,review
0,9781542040464,5,Terrific purchase,"Chetan Bhagat sir. Seriously this is your best work. The way you represented an unlove story & a murder mystery is worth reading. The characters, dialogues, locations & everything about the book is amazing. And yes I can say now that 'The girl is room number 105' is my favourite book I've read so far. The story shows real faces of people which is hidden behind several layers. Finally, I enjoyed a lot & would feel great to recommend the same to other readers. And hope more interesting books fr..."
1,9781542040464,5,AWESOMENESS IN ITS OWN SWAG,one must be good enough to not let someone die just the thing one couldnt imagine that chetan bhagat could also do this stufflast pages were just like thrill making my spine shiver ONE MORE AWESOME STORY BY CHETAN BHAGAT TO INDIAN READERS INDEED THE PAPERBACK KING LOVE FROM INDIA
2,9781542040464,5,Awesome,Loved it. It seemed like a de ja vu for me. We broke up in Oct 2016 after a 5year long intense relationship. Since then I couldn't love anybody else. Hope this unlove story will finally bring back some love inside my heart.... 😒😁
3,9781542040464,4,Really Nice,"Okay so TBH this is my second read of Chetan Bhagat's novel after One Indian Girl. The story is very engaging . The plot and twist was good . One thing I felt while reading the novel(Also happened with the previous that I read),I could able to create a imaginary scenes in my mind,like movies ! There is some character like Keshav,Saurabh,That Gutkha Chewing Chandan Arora , I can relate with them as I also have seen this types of people like their nature and all although I can realate actually ..."
4,9781542040464,4,Delightful,"In my opinion, this is the 3rd best book of the writer right after 'Five Points Someone' and 'Revolution 2020'. This is an unputdownable mystery thriller. Not the best in this suspense category. If you are advance level reader and have already read Sidney Sheldon or Dan Brown and you have little common sense then you can easily guess who the killer is among all the skeptical charecters. In other words the plot is just so simple. It goes like this, the girl, here the victim, gets killed and he..."


,product_id,rating,summary,review
3285,9781542040464,5,Awesome,"LITERALLY FANTASTIC STORY,,, I LOVE CHETAN SIR,,"
3286,9781542040464,4,Wonderful,looks good
3287,9781542040464,5,Best in the market!,This one's a must read! Such a thrilling story..Finished half the book in just a couple of hours and can't wait to read the rest!
3288,9781542040464,5,Brilliant,damn interesting!!! I loved the book..
3289,9781542040464,5,Fabulous!,It's always good to have a book of Chetan Bhagat simply owsome stories we had gone through now looking for more enjoyable concept.


In [ ]:
# DS-8

In [ ]:
link = ['https://www.flipkart.com/one-arranged-murder/product-reviews/itmb4cd0197e98ce?pid=9781542094139&lid=LSTBOK9781542094139A5FY6R&marketplace=FLIPKART']
dataset_8 = []

for idx, url in enumerate(tqdm(link, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,5000+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(2)
        soup = BeautifulSoup(r.content, 'html.parser') 
        
        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()

            # appending to data
            dataset_8.append({'product_id':pid, 'rating': rating, 'summary': summary, 'review': review})

In [ ]:
df8 = pd.DataFrame(dataset_8)

with pd.option_context('display.max_colwidth', -1):
    display(df8.head(5))
    display(df8.tail(5))

df8.to_csv('/Users/kritya/Desktop/College/SEM 7/Project Phase 1/DS/FYP - FlipBook8.csv')